In [1]:
import sys
import os
import warnings
import numpy as np
import tensorflow as tf
import SimpleITK as sitk
from nilearn import image, plotting
import nibabel as nib

sys.path.append("../")
sys.path.append("../deepneuroan")
warnings.filterwarnings('ignore')

from deepneuroan.data_generator import DataGenerator
from deepneuroan.models import ChannelwiseConv3D, rigid_concatenated
from deepneuroan.generate_train_data import transform_volume
from deepneuroan.preproc import create_ref_grid, get_mni152_affine_nib, get_mni152_affine_itk

/home/ltetrel/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
E0121 11:58:16.193799 140396591261504 due.py:63] Failed to import duecredit due to No module named 'duecredit'


In [2]:
data_dir = "/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/"
template_filepath = os.path.join(data_dir, "template_on_grid")

list_files = []
list_files_tmp = set([])
for root, _, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file).split('.')[0]
        if os.path.exists(filepath + ".txt"):
            list_files_tmp.add(filepath)
list_files = list(list_files_tmp)

bs = 2
ncpu = 1
np.random.seed(0)
params_gen = dict(list_files=list_files, template_file=template_filepath, batch_size=bs, avail_cores=ncpu)
valid_gen = DataGenerator(partition="valid", **params_gen)

In [3]:
with open("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/rigid_concatenated_2020-01-17_22:26:33.json", "r") as json_file:
    model = tf.keras.models.model_from_json(json_file.read(), custom_objects={'ChannelwiseConv3D': ChannelwiseConv3D})
model.load_weights("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/checkpoints/rigid_concatenated_best/rigid_concatenated_cp-0250.ckpt")
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01)
                  , loss=tf.keras.losses.mean_squared_error
                  , metrics=["mae"])

In [4]:
ref_grid = create_ref_grid()

# tmp
batch = 0

moving = valid_gen.__getitem__(0)[0][:, :, :, :,0][batch,]
fixed = valid_gen.__getitem__(0)[0][:, :, :, :,1][batch,]
pred = sitk.GetImageFromArray(moving)
pred.SetOrigin(ref_grid.GetOrigin())
pred.SetSpacing(ref_grid.GetSpacing())
pred.SetDirection(ref_grid.GetDirection())
# t_rigid = model.predict(x=valid_gen.__getitem__(0)[0], use_multiprocessing=False, verbose=0)[batch,]
t_rigid = valid_gen.__getitem__(0)[1][batch,]
registered_pred = transform_volume(pred, ref_grid, interp=sitk.sitkBSplineResampler, rigid=t_rigid)
pred_mask = sitk.GetArrayFromImage(registered_pred) > 0
truth_mask = fixed > 0
# for i in range(all_gen.__len__()):
#     fixed = valid_gen.__getitem__(0)[0][:, :, :, :,0]
#     moving = valid_gen.__getitem__(0)[0][:, :, :, :,1]
#     for batch in range(valid_gen.batch_size):
#         sitk.GetImageFromArray(valid_gen.__getitem__(0)[0][:, :, :, :,0])
#         pred = sitk.Image(ref_grid.GetSize(), ref_grid.GetPixelIDValue())
#         pred.SetOrigin(ref_grid.GetOrigin())
#         pred.SetSpacing(ref_grid.GetSpacing())
#         pred.SetDirection(ref_grid.GetDirection())

In [5]:
sitk.WriteImage(registered_pred, "registered_pred.nii.gz")
tmp = sitk.GetImageFromArray(fixed)
tmp.SetOrigin(ref_grid.GetOrigin())
tmp.SetSpacing(ref_grid.GetSpacing())
tmp.SetDirection(ref_grid.GetDirection())
sitk.WriteImage(tmp, "fixed.nii.gz")

In [6]:
def dice_loss(y_true, y_pred):
  
    numerator = 2 * np.sum(y_true * y_pred)
    denominator = np.sum(y_true) + np.sum(y_pred)
    print(numerator)
    print(denominator)
    
    return (numerator + 1) / (denominator + 1)

dice_loss(truth_mask, pred_mask)

3747738
3908808


0.9587930748215121

In [10]:
nib_img = nib.load("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/template_on_grid.nii.gz")
nib_img.get_affine()

array([[  -1.,    0.,    0.,  109.],
       [   0.,    1.,    0., -127.],
       [   0.,    0.,    1.,  -91.],
       [   0.,    0.,    0.,    1.]])

In [17]:
itk_img = sitk.ReadImage("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/template_on_grid.nii.gz", sitk.sitkFloat32)
print(itk_img.GetDirection())
print(itk_img.GetSpacing())
print(itk_img.GetOrigin())

(1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(1.0, 1.0, 1.0)
(-109.0, 127.0, -91.0)


In [ ]:
# fixed = nib.load("/home/ltetrel/Documents/work/DeepNeuroAN/template/MNI152_T1_1mm_brain.nii.gz")
fixed = nib.Nifti1Image(fixed, get_mni152_affine_nib())
pred = nib.Nifti1Image(registered_pred, get_mni152_affine_nib())

In [ ]:
display = plotting.plot_anat(fixed)
display.add_edges(pred)